# Intro 2 : Complex Data Structure

Nested Data Structure is common for API calls, but they are annoying to handle. `LionAGI` has some powerful functions to handle nested data structures.

In [1]:
from lionagi.libs import nested

In [2]:
# let us define a couple nested structure

nested_list = [0, [1, 2, [3, 4]]]

nested_dict = {"level1": {"level2": {"level3": "some_value"}}}

## 1. Nested Structure Operations
- nset
- nget
- is_structure_homogenous

`nset` & `nget`

suppose you have a nested dictionary as above, how would you access the value? typically you can do the following,

In [3]:
value1 = nested_dict["level1"]["level2"]["level3"]

but this can be tiresome, especially after the keys get long and with confusing names 

`nget` - nested get

allows you to extract the value in the nested structure using a list

In [4]:
keys = ["level1", "level2", "level3"]

value2 = nested.nget(nested_dict, keys)

In [5]:
value1 == value2

True

similarly you can use `nset` to set a value in a nested structure

In [6]:
nested.nset(nested_dict, keys, "new_value")
print("Nested dictionary after setting value:")

Nested dictionary after setting value:


`nset` and `nget` work for both nested lists and nested dictionary

In [7]:
indices = [1, 2, 1]
nested.nget(nested_list, indices)

4

In [8]:
# Setting a new value at a specific index in the nested list
nested.nset(nested_list, indices, "new_value")
print("Nested list after setting value:", nested_list)

Nested list after setting value: [0, [1, 2, [3, 'new_value']]]


if you try to get a value of an non-existent path, default value is `None`

In [9]:
nested_dict = {"level1": {"level2": {"level3": "value"}}}

# Attempting to get a value at a non-existent path,
value = nested.nget(
    nested_dict, ["level1", "level2", "non_existent_key"], default="hello"
)
print("Value from default:", value)

Value from default: hello


## 2. Flattening and Unflattening a complex nested data structure
- flatten
- unflatten

we can also flatten a dictionary of complex nested types

In [10]:
nested_dict = {
    "a": [1, 2, 3],
    "b": 4,
    "c": [{"d": 5, "e": 6}, {"f": 7, "g": 8}],
}

In [11]:
# now we can flatten it to a 1-d dictionary
flattened_dict = nested.flatten(nested_dict)
print(flattened_dict)

{'a[^_^]0': 1, 'a[^_^]1': 2, 'a[^_^]2': 3, 'b': 4, 'c[^_^]0[^_^]d': 5, 'c[^_^]0[^_^]e': 6, 'c[^_^]1[^_^]f': 7, 'c[^_^]1[^_^]g': 8}


In [12]:
# Flatten the nested dictionary with a max depth of 1
flat_dict = nested.flatten(nested_dict, max_depth=1)
print("Flattened nested dictionary with max depth:", flat_dict)

Flattened nested dictionary with max depth: {'a[^_^]0': 1, 'a[^_^]1': 2, 'a[^_^]2': 3, 'b': 4, 'c[^_^]0': {'d': 5, 'e': 6}, 'c[^_^]1': {'f': 7, 'g': 8}}


if you just want the keys as path, you can use `get_flattened_keys`

In [13]:
keys = nested.get_flattened_keys(nested_dict)

print(keys)
print(f"\nNumber of unique key-value pair: {len(keys)}")

['a[^_^]0', 'a[^_^]1', 'a[^_^]2', 'b', 'c[^_^]0[^_^]d', 'c[^_^]0[^_^]e', 'c[^_^]1[^_^]f', 'c[^_^]1[^_^]g']

Number of unique key-value pair: 8


Let's say you have done some operations on the flattened dictionary, and you would like to fold them into an organized nested structure, 

you can use `unflatten`

In [14]:
unflattened_dict = nested.unflatten(flattened_dict)

print(unflattened_dict)
print(unflattened_dict == nested_dict)

{'a': [1, 2, 3], 'b': 4, 'c': [{'d': 5, 'e': 6}, {'f': 7, 'g': 8}]}
True


## 3. ninsert, nfilter and nmerge

you can insert value into a nested dictionary according to the path, if path doesn't exist, `ninsert` will create it

In [15]:
# create a nested structure by inserting value, key will be created as a nested manner
nested_dict = {}
nested.ninsert(nested_dict, ["a", "b", "c"], value=1)
nested.ninsert(nested_dict, ["a", "b", "d"], value=2)
nested.ninsert(nested_dict, ["a", "e"], value=3)
nested.ninsert(nested_dict, ["f"], value=4)
print(nested_dict)

{'a': {'b': {'c': 1, 'd': 2}, 'e': 3}, 'f': 4}


`nfilter` can be used to filter out elements of a nested structure

In [16]:
nested_dict1 = {
    "data": {"temperature": 22, "humidity": 80, "pressure": 1012},
    "threshold": {"temperature": 20, "humidity": 85, "pressure": 1000},
}


# finding value larger than threshold
def condition_for_dict1(item):
    key, value = item
    return value > nested_dict1["threshold"][key]


filtered_data1 = nested.nfilter(nested_dict1["data"], condition_for_dict1)
print("Filtered nested dictionary:", filtered_data1)

Filtered nested dictionary: {'temperature': 22, 'pressure': 1012}


In [17]:
list_of_dicts = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25},
    {"name": "Charlie", "age": 35},
]


# finding people with age larger than 30
def condition_for_list_of_dicts(item):
    return item.get("age", 0) > 30


filtered_list_of_dicts = nested.nfilter(
    list_of_dicts, condition_for_list_of_dicts
)
print("Filtered list of dictionaries:", filtered_list_of_dicts)

Filtered list of dictionaries: [{'name': 'Charlie', 'age': 35}]


we can also merge different data structure

In [18]:
# Usage Example 1: Merging dictionaries with overwriting
dicts_to_merge = [{"a": 1, "b": 2}, {"b": 3, "c": 4}]

merged_dict = nested.nmerge(dicts_to_merge, overwrite=True)
print("Merged dictionaries with overwriting:", merged_dict)

Merged dictionaries with overwriting: {'a': 1, 'b': 3, 'c': 4}


In [21]:
# Usage Example 2: Merging dictionaries without overwriting
dicts_to_merge = [{"a": 1, "b": 2}, {"b": 3, "c": 4}]

# creating unique keys for duplicates
merged_dict = nested.nmerge(
    dicts_to_merge,
    overwrite=False,
    dict_sequence=True,
)
print("Merged dictionaries without overwriting:", merged_dict)

Merged dictionaries without overwriting: {'a': 1, 'b': 2, 'b[^_^]1': 3, 'c': 4}


In [20]:
# Usage Example 3: Merging lists with sorting
lists_to_merge = [[3, 1], [4, 2]]

# Merge lists and sort the result
merged_list = nested.nmerge(lists_to_merge, sort_list=True)
print("Merged and sorted lists:", merged_list)

Merged and sorted lists: [1, 2, 3, 4]
